In [ ]:
pip install atproto requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.4/380.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.1/301.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 0.8.3 requires websockets>=14.0, but you have websockets 13.1 which is incompatible.


In [ ]:
from atproto import Client
import re
import atproto
import json

In [ ]:
USERNAME = ""
PASSWORD = ""

client = Client()
client.login(USERNAME, PASSWORD)
print("Login bem-sucedido!")


Login bem-sucedido!


In [ ]:
import json

def get_post_details(post):
    return {
        "text": post.record.text or "",
        "Data_criacao": post.record.created_at or ""
    }

def search_posts_multiple(client, keywords, limit=3000):
    posts = []
    seen_post_ids = set()
    for keyword in keywords:
        cursor = None
        while len(posts) < limit:
            params = {
                'q': keyword,
                'limit': min(50, limit - len(posts))
            }
            if cursor:
                params['cursor'] = cursor

            response = client.app.bsky.feed.search_posts(params)

            for post in response.posts:
                if post.uri not in seen_post_ids:
                    posts.append(post)
                    seen_post_ids.add(post.uri)

            if not response.cursor:
                break
            cursor = response.cursor
    return posts[:limit]

def main():
    keywords = ["Mercado Livre", "mercado livre"]
    raw_posts = search_posts_multiple(client, keywords, limit=3000)

    print(f"Posts buscados: {len(raw_posts)}")

    post_details_list = [get_post_details(post) for post in raw_posts]

    output_file = "posts_mercado_livre_details.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(post_details_list, f, indent=4, ensure_ascii=False)

    print(f"Salvo {len(post_details_list)} postagens em {output_file}")

if __name__ == "__main__":
    main()


Posts buscados: 3000
Salvo 3000 postagens em posts_mercado_livre_details.json


In [ ]:
with open("posts_mercado_livre_details.json", "r", encoding="utf-8") as f:
    posts = json.load(f)

print(f"Posts originais: {len(posts)}")

def filtrar_por_texto(lista_posts):
    return [post for post in lista_posts if post.get("text") not in (None, "")]

posts_filtrados = filtrar_por_texto(posts)

print(f"Posts após filtro por texto: {len(posts_filtrados)}")

with open("posts_mercado_livre_filtrados.json", "w", encoding="utf-8") as f:
    json.dump(posts_filtrados, f, indent=4, ensure_ascii=False)


Posts originais: 3000
Posts após filtro por texto: 3000


In [ ]:
import json
import requests

# 1. Caminho do arquivo JSON local
json_path = "/content/posts_mercado_livre_filtrados.json"

# 2. URL gerada pelo Power Automate
# Substitua pela URL do seu gatilho "Quando uma solicitação HTTP é recebida"
power_automate_url = "https://prod-187.westus.logic.azure.com:443/workflows/834fb8eeb28d4a8bb026f3dd20da5ad2/triggers/manual/paths/invoke?api-version=2016-06-01&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=Fxf91NKyr7khMTD8H0fCfzOT32Wd0d2lVFHcpPgh-BQ"

# 3. Ler o arquivo JSON
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 4. Enviar para o Power Automate
headers = {"Content-Type": "application/json"}
response = requests.post(power_automate_url, headers=headers, json=data)

# 5. Resultado
if response.status_code == 200 or response.status_code == 202:
    print("✅ Dados enviados com sucesso!")
else:
    print(f"❌ Erro ao enviar: {response.status_code} - {response.text}")


✅ Dados enviados com sucesso!
